In [39]:
import numpy as np
import pandas as pd
import cv2
import os
import pathlib 
from matplotlib import pyplot as plt
from import_train_test import Import_path_to_data

In [40]:
train_path, test_path = Import_path_to_data.Import_train_test()
folder_list = os.listdir(train_path)

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        img = cv2.cvtColor(cv2.imread(os.path.join(folder_path, filename)), cv2.COLOR_BGR2RGB)
        if img is not None:
            images.append(img)
    return images

def list_the_img(train_path):    
    list_off_img = []
    for class_ in folder_list:
        path_ = pathlib.Path(f'{train_path}\{class_}')
        img = load_images_from_folder(path_)
        list_off_img.append(img)
    return list_off_img

def X_y(list_off_img): 
    X = []
    y = []

    for index_1 in range(0, len(list_off_img)):
        for index_2 in list_off_img[index_1]:
            X.append(index_2)
            y.append(folder_list[index_1])
            
    return X, y

In [41]:
X_train_raw, y_train = X_y(list_the_img(train_path))
X_test_raw, y_test = X_y(list_the_img(test_path))

In [42]:
X_train = np.expand_dims(X_train_raw, axis=-1).astype(np.float32) / 255
X_test = np.expand_dims(X_test_raw, axis=-1).astype(np.float32) / 255

In [43]:
import tensorflow as tf
import keras
from keras.layers import Conv2D, Dense, MaxPool2D, Flatten, Dropout
from keras import Sequential
from keras.activations import relu, softmax
from keras.optimizers import Nadam
from keras.initializers import he_normal
from keras.losses import sparse_categorical_crossentropy
from keras.backend import clear_session
from tensorflow.random import set_seed
from functools import partial

In [44]:
clear_session()
set_seed(42)

In [45]:
DefaultConv2D = partial(Conv2D, kernel_size=3, padding="same",
                        activation=relu, kernel_initializer=he_normal)
model = Sequential([
    DefaultConv2D(filters=64, kernel_size=7, input_shape=[100, 100, 4]),
    MaxPool2D(),
    DefaultConv2D(filters=128),
    DefaultConv2D(filters=128),
    MaxPool2D(),
    DefaultConv2D(filters=256),
    DefaultConv2D(filters=256),
    MaxPool2D(),
    Flatten(),
    Dense(units=128, activation=relu,
                          kernel_initializer=he_normal),
    Dropout(0.5),
    Dense(units=64, activation=relu,
                          kernel_initializer=he_normal),
    Dropout(0.5),
    Dense(units=10, activation=softmax)
])

optimizer = Nadam(learning_rate=0.1e-3,)
model.compile(loss=sparse_categorical_crossentropy, optimizer=optimizer,
              metrics=['accuracy'])

In [46]:
history = model.fit(X_train, y_train, epochs=15,
                    validation_batch_size=0.1)
score = model.evaluate(X_test, y_test)
X_new = X_test[:10]
y_pred = model.predict(X_new)

ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'str'>"})